# Logistic regression on employment outcomes

#### Import packages

In [415]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from prettytable import PrettyTable

#### Load dataset

In [416]:
df = pd.read_excel("full_cleaned_survey.xlsx")

In [417]:
df.head(5)

,id,disability,employment_status,type_of_disability,disability_obtained,gender,race,age,urban_rural,level_of_education,household_income,duration_unemployed,previous_employment,q1_response,q1_themes,q2_response,q2_themes,assistive_tech
0,1,Yes,unemployed,Visual impairment,born,Male,African (Black),29,urban,secondary,0 - 5 000,6 months to 1 year,Yes,No,"[""NO""]",My eyesight,"[""DIS""]",No
1,2,Yes,unemployed,Visual impairment,born,Female,African (Black),27,rural,secondary,0 - 5 000,2 years to 2.5 years,No,Yes. The lack of awareness with regards to whi...,"[""LA""]",There are only a few employment opportunities ...,"[""ACC""]",No
2,3,Yes,unemployed,"Physical disability, Hearing impairment",born,Male,Indian,45,urban,diploma,15 001 - 20 000,4.5 years to 5 years,Yes,No,"[""NO""]",Being too old. Being discriminated against.,"[""DEMO"",""DISC""]",Yes
3,4,Yes,unemployed,Visual impairment,developed,Female,Caucasian (White),36,urban,secondary,5 001 - 10 000,1 year to 1.5 years,Yes,"Yes, it has made me where I want to be, but th...","[""RA""]",My disabilities and my age. Due I won't be com...,"[""ACC""]",Yes
4,5,Yes,unemployed,Physical disability,developed,Male,African (Black),25,rural,vocational,0 - 5 000,1 year to 1.5 years,No,Currently I'm in search of in-service training...,"[""AC""]",I feel that having one hand has brought diffic...,"[""DIS""]",No


### Converted into ordinals

Level of education

duration unemployed

household income

In [418]:
df.columns

Index(['id', 'disability', 'employment_status', 'type_of_disability',
       'disability_obtained', 'gender', 'race', 'age', 'urban_rural',
       'level_of_education', 'household_income', 'duration_unemployed',
       'previous_employment', 'q1_response', 'q1_themes', 'q2_response',
       'q2_themes', 'assistive_tech'],
      dtype='object')

#### Duration unemployed to ordinal

In [419]:
print(df['duration_unemployed'].unique())

['6 months to 1 year' '2 years to 2.5 years' '4.5 years to 5 years'
 '1 year to 1.5 years' '1.5 years to 2 years' '6 years to 6.5 years'
 '7.5 years to 8 years' '3 years to 3.5 years' '5 years to 5.5 years'
 'Less than 3 months' '6.5 years to 7 years' '3 to 6 months'
 '2.5 years to 3 years' '4 years to 4.5 years' '7 years to 7.5 years'
 '5.5 years to 6 years' '3.5 years to 4 years']


#### Group the periods together into years

In [420]:
# Function to categorize duration into logical groups
def simplify_duration(duration):
    if duration in ['Less than 3 months', '0', '3 to 6 months', '6 months to 1 year']:
        return '0-1 years'
    elif duration in ['1 year to 1.5 years', '1.5 years to 2 years']:
        return '1-2 years'
    elif duration in ['2 years to 2.5 years', '2.5 years to 3 years']:
        return '2-3 years'
    elif duration in ['3 years to 3.5 years', '3.5 years to 4 years']:
        return '3-4 years'
    elif duration in ['4 years to 4.5 years', '4.5 years to 5 years']:
        return '4-5 years'
    elif duration in ['5 years to 5.5 years', '5.5 years to 6 years']:
        return '5-6 years'
    elif duration in ['6 years to 6.5 years', '6.5 years to 7 years']:
        return '6-7 years'
    elif duration in ['7 years to 7.5 years', '7.5 years to 8 years']:
        return '7-8 years'
    else:
        print(duration)
        return 'Other'

# Create new column
df['duration_unemployed'] = df['duration_unemployed'].apply(simplify_duration)

In [421]:
df["duration_unemployed"].value_counts()

duration_unemployed
7-8 years    27
0-1 years    24
1-2 years    18
2-3 years    13
6-7 years    12
4-5 years     8
3-4 years     8
5-6 years     8
Name: count, dtype: int64

#### Mapping into ordinal

In [422]:
# Define the mapping for 'duration_unemployed'
duration_mapping = {
    '0-1 years': 0,
    '1-2 years': 1,
    '2-3 years': 2,
    '3-4 years': 3,
    '4-5 years': 4,
    '5-6 years': 5,
    '6-7 years': 6,
    '7-8 years': 7
    }

# Apply the mapping and convert the result to integer
df['duration_unemployed_ordinal'] = df['duration_unemployed'].map(duration_mapping)

# Check the result
print(df[['duration_unemployed', 'duration_unemployed_ordinal']])


    duration_unemployed  duration_unemployed_ordinal
0             0-1 years                            0
1             2-3 years                            2
2             4-5 years                            4
3             1-2 years                            1
4             1-2 years                            1
..                  ...                          ...
113           2-3 years                            2
114           1-2 years                            1
115           2-3 years                            2
116           6-7 years                            6
117           0-1 years                            0

[118 rows x 2 columns]


In [423]:
df.head(5)

,id,disability,employment_status,type_of_disability,disability_obtained,gender,race,age,urban_rural,level_of_education,household_income,duration_unemployed,previous_employment,q1_response,q1_themes,q2_response,q2_themes,assistive_tech,duration_unemployed_ordinal
0,1,Yes,unemployed,Visual impairment,born,Male,African (Black),29,urban,secondary,0 - 5 000,0-1 years,Yes,No,"[""NO""]",My eyesight,"[""DIS""]",No,0
1,2,Yes,unemployed,Visual impairment,born,Female,African (Black),27,rural,secondary,0 - 5 000,2-3 years,No,Yes. The lack of awareness with regards to whi...,"[""LA""]",There are only a few employment opportunities ...,"[""ACC""]",No,2
2,3,Yes,unemployed,"Physical disability, Hearing impairment",born,Male,Indian,45,urban,diploma,15 001 - 20 000,4-5 years,Yes,No,"[""NO""]",Being too old. Being discriminated against.,"[""DEMO"",""DISC""]",Yes,4
3,4,Yes,unemployed,Visual impairment,developed,Female,Caucasian (White),36,urban,secondary,5 001 - 10 000,1-2 years,Yes,"Yes, it has made me where I want to be, but th...","[""RA""]",My disabilities and my age. Due I won't be com...,"[""ACC""]",Yes,1
4,5,Yes,unemployed,Physical disability,developed,Male,African (Black),25,rural,vocational,0 - 5 000,1-2 years,No,Currently I'm in search of in-service training...,"[""AC""]",I feel that having one hand has brought diffic...,"[""DIS""]",No,1


### Household income into ordinal

In [424]:
df['household_income'].value_counts()

household_income
0 - 5 000          71
5 001 - 10 000     22
10 001 - 15 000    11
15 001 - 20 000     6
40 001 - 45 000     2
20 001 - 25 000     2
25 001 - 30 000     2
30 001 - 35 000     2
Name: count, dtype: int64

In [425]:
df['household_income'].unique()

array(['0 - 5 000', '15 001 - 20 000', '5 001 - 10 000',
       '40 001 - 45 000', '10 001 - 15 000', '20 001 - 25 000',
       '25 001 - 30 000', '30 001 - 35 000'], dtype=object)

### mapping into ordinal

In [426]:
income_mapping = {
    '0 - 5 000': 0,
    '5 001 - 10 000': 1,
    '10 001 - 15 000': 2,
    '15 001 - 20 000': 3,
    '20 001 - 25 000': 4,
    '25 001 - 30 000': 5,
    '30 001 - 35 000': 6,
    '35 001 - 40 000': 7,
    '40 001 - 45 000': 8
}

# Apply the mapping
df['household_income_ordinal'] = df['household_income'].map(income_mapping)

# Check the result
print(df[['household_income', 'household_income_ordinal']])

    household_income  household_income_ordinal
0          0 - 5 000                         0
1          0 - 5 000                         0
2    15 001 - 20 000                         3
3     5 001 - 10 000                         1
4          0 - 5 000                         0
..               ...                       ...
113   5 001 - 10 000                         1
114        0 - 5 000                         0
115        0 - 5 000                         0
116        0 - 5 000                         0
117  10 001 - 15 000                         2

[118 rows x 2 columns]


### Level of Education into ordinal

In [427]:
df['level_of_education'].value_counts()

level_of_education
secondary              50
undergraduate          19
masters                15
vocational             12
diploma                10
honours                 6
no formal education     4
phd                     2
Name: count, dtype: int64

In [428]:
# Replace "no formal education" with "none"
df['level_of_education'] = df['level_of_education'].replace('no formal education', 'none')

# Define the ranking for education levels
education_mapping = {
    'none': 0,
    'secondary': 1,
    'vocational': 2,
    'diploma': 3,
    'undergraduate': 4,
    'honours': 5,
    'masters': 6,
    'phd': 7
}

# Apply the mapping
df['education_ordinal'] = df['level_of_education'].map(education_mapping)

# Check the result
print(df[['level_of_education', 'education_ordinal']].head(10))


  level_of_education  education_ordinal
0          secondary                  1
1          secondary                  1
2            diploma                  3
3          secondary                  1
4         vocational                  2
5          secondary                  1
6          secondary                  1
7          secondary                  1
8          secondary                  1
9          secondary                  1


# Notes to consider for columns

Removals: disability, q1_response, q2_response

one hot and label encoding:

race

type_of_disability



binarize:

gender

previous_employment

urban_rural



In [429]:
df.columns

Index(['id', 'disability', 'employment_status', 'type_of_disability',
       'disability_obtained', 'gender', 'race', 'age', 'urban_rural',
       'level_of_education', 'household_income', 'duration_unemployed',
       'previous_employment', 'q1_response', 'q1_themes', 'q2_response',
       'q2_themes', 'assistive_tech', 'duration_unemployed_ordinal',
       'household_income_ordinal', 'education_ordinal'],
      dtype='object')

### removals

In [430]:
df = df.drop(columns=['disability', 'q1_response', 'q2_response'])

### Name changes for easier use

In [431]:
df = df.rename(columns={'type_of_disability': 'disability', 'urban_rural': 'location', 'level_of_education': 'education', 'disability_obtained': 'obtained', 'previous_employment': 'experience'})

types of disabilities, some people have more than 1

In [432]:
df['disability'].value_counts()

disability
Physical disability                        49
Visual impairment                          40
Hearing impairment                         12
Mental health condition                     6
Physical disability, Visual impairment      5
Physical disability, Hearing impairment     3
Epilepsy                                    3
Name: count, dtype: int64

In [433]:
df

,id,employment_status,disability,obtained,gender,race,age,location,education,household_income,duration_unemployed,experience,q1_themes,q2_themes,assistive_tech,duration_unemployed_ordinal,household_income_ordinal,education_ordinal
0,1,unemployed,Visual impairment,born,Male,African (Black),29,urban,secondary,0 - 5 000,0-1 years,Yes,"[""NO""]","[""DIS""]",No,0,0,1
1,2,unemployed,Visual impairment,born,Female,African (Black),27,rural,secondary,0 - 5 000,2-3 years,No,"[""LA""]","[""ACC""]",No,2,0,1
2,3,unemployed,"Physical disability, Hearing impairment",born,Male,Indian,45,urban,diploma,15 001 - 20 000,4-5 years,Yes,"[""NO""]","[""DEMO"",""DISC""]",Yes,4,3,3
3,4,unemployed,Visual impairment,developed,Female,Caucasian (White),36,urban,secondary,5 001 - 10 000,1-2 years,Yes,"[""RA""]","[""ACC""]",Yes,1,1,1
4,5,unemployed,Physical disability,developed,Male,African (Black),25,rural,vocational,0 - 5 000,1-2 years,No,"[""AC""]","[""DIS""]",No,1,0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
113,114,employed,Physical disability,developed,Male,African (Black),42,urban,vocational,5 001 - 10 000,2-3 years,Yes,"[""RA""]",[],No,2,1,2
114,115,employed,Physical disability,born,Female,African (Black),31,rural,secondary,0 - 5 000,1-2 years,Yes,"[""NO""]","[""SYS""]",No,1,0,1
115,116,employed,Visual impairment,developed,Female,African (Black),31,urban,secondary,0 - 5 000,2-3 years,No,"[""YES""]",[],No,2,0,1
116,117,employed,Hearing impairment,born,Male,African (Black),36,urban,secondary,0 - 5 000,6-7 years,Yes,"[""LA""]",[],Yes,6,0,1


### Binary label encoding

In [434]:
# List of columns to binarize
binary_columns = ['gender', 'experience', 'assistive_tech']

# Binarize the gender, experience, and assistive_tech columns manually
df['gender'] = df['gender'].map({'Male': 0, 'Female': 1})  # Example for gender (adjust if needed)
df['experience'] = df['experience'].map({'No': 0, 'Yes': 1})  # Example for experience
df['assistive_tech'] = df['assistive_tech'].map({'No': 0, 'Yes': 1})  # Example for assistive_tech
df['location'] = df['location'].map({'rural': 0, 'urban': 1})  # Example for assistive_tech

### One and multi hot encoding

Race

In [435]:
# Apply one-hot encoding to location and race columns
df = pd.get_dummies(df, columns=['race'], prefix=['race'])
df = df.drop(columns=['race_Caucasian (White)'])

# Convert any boolean columns to 1 and 0
df = df.applymap(lambda x: 1 if x is True else (0 if x is False else x))

# Display the modified dataframe
print(df.head())

   id employment_status                               disability   obtained  \
0   1        unemployed                        Visual impairment       born   
1   2        unemployed                        Visual impairment       born   
2   3        unemployed  Physical disability, Hearing impairment       born   
3   4        unemployed                        Visual impairment  developed   
4   5        unemployed                      Physical disability  developed   

   gender  age  location   education household_income duration_unemployed  \
0       0   29         1   secondary        0 - 5 000           0-1 years   
1       1   27         0   secondary        0 - 5 000           2-3 years   
2       0   45         1     diploma  15 001 - 20 000           4-5 years   
3       1   36         1   secondary   5 001 - 10 000           1-2 years   
4       0   25         0  vocational        0 - 5 000           1-2 years   

   experience q1_themes        q2_themes  assistive_tech  \
0 

C:\Users\liand\AppData\Local\Temp\ipykernel_17824\3018448151.py:6: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: 1 if x is True else (0 if x is False else x))


Type of disability

In [436]:
# Split the disabilities in the 'disability' column by commas and strip spaces
df['disability'] = df['disability'].apply(lambda x: x.split(', ') if isinstance(x, str) else [])

# Get the unique disabilities in the dataset and replace spaces with underscores
all_disabilities = set([disability.replace(' ', '_') for sublist in df['disability'] for disability in sublist])

# Create binary columns for each disability type
for disability in all_disabilities:
    df[disability] = df['disability'].apply(lambda x: 1 if disability.replace('_', ' ') in x else 0)

# Drop the original 'disability' column if you no longer need it
df = df.drop(columns=['disability'])

In [437]:
df = df.rename(columns={'race_African (Black)': 'race_African', 'race_Caucasian (White)': 'race_Caucasian'})

In [438]:
df

,id,employment_status,obtained,gender,age,location,education,household_income,duration_unemployed,experience,...,household_income_ordinal,education_ordinal,race_African,race_Coloured,race_Indian,Epilepsy,Physical_disability,Visual_impairment,Mental_health_condition,Hearing_impairment
0,1,unemployed,born,0,29,1,secondary,0 - 5 000,0-1 years,1,...,0,1,1,0,0,0,0,1,0,0
1,2,unemployed,born,1,27,0,secondary,0 - 5 000,2-3 years,0,...,0,1,1,0,0,0,0,1,0,0
2,3,unemployed,born,0,45,1,diploma,15 001 - 20 000,4-5 years,1,...,3,3,0,0,1,0,1,0,0,1
3,4,unemployed,developed,1,36,1,secondary,5 001 - 10 000,1-2 years,1,...,1,1,0,0,0,0,0,1,0,0
4,5,unemployed,developed,0,25,0,vocational,0 - 5 000,1-2 years,0,...,0,2,1,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
113,114,employed,developed,0,42,1,vocational,5 001 - 10 000,2-3 years,1,...,1,2,1,0,0,0,1,0,0,0
114,115,employed,born,1,31,0,secondary,0 - 5 000,1-2 years,1,...,0,1,1,0,0,0,1,0,0,0
115,116,employed,developed,1,31,1,secondary,0 - 5 000,2-3 years,0,...,0,1,1,0,0,0,0,1,0,0
116,117,employed,born,0,36,1,secondary,0 - 5 000,6-7 years,1,...,0,1,1,0,0,0,0,0,0,1


In [439]:
df['employment_status'].head(5)

0    unemployed
1    unemployed
2    unemployed
3    unemployed
4    unemployed
Name: employment_status, dtype: object

In [440]:
df['employment_status'] = df['employment_status'].map({'unemployed': 1, 'employed': 0})

### Qual question themes

Q1

In [441]:
df['q1_themes'].unique()

array(['["NO"]', '["LA"]', '["RA"]', '["AC"]', '["YES"]', '["RA", "DI"]',
       '["RA", "DI", "LC"]', '["RA", "DI", "LC", "NR"]',
       '["RA","DI","LC","NR"]', '["YES","AC"]', '["RA","DI","NR"]', '[]',
       '["RA","DI"]', '["RA","AC"]', '["DI"]', '["DI","NR"]',
       '["RA","AC","DI"]', '["LA","DI"]', '["LC"]', '["AC","RA"]',
       '["NO"] '], dtype=object)

In [442]:
import ast 

# First clean up the string format and convert to actual lists
df['q1_themes'] = df['q1_themes'].apply(lambda x: ast.literal_eval(x.replace(' ','')))

# Use explode to split combined themes into separate rows
df_exploded = df.explode('q1_themes')

In [443]:
# Create crosstab with percentages (normalize='index' for row percentages)
crosstab_pct = pd.crosstab(df_exploded['education'], df_exploded['q1_themes'], normalize='index') * 100

# Round to 1 decimal place
crosstab_pct = crosstab_pct.round(2)

print(crosstab_pct)

q1_themes         AC     DI     LA     LC     NO     NR     RA    YES
education                                                            
diploma        7.690  7.690  0.000  7.690 46.150  7.690 23.080  0.000
honours        0.000 27.270  0.000 18.180  9.090  0.000 36.360  9.090
masters        0.000 43.330  3.330  0.000  0.000 16.670 36.670  0.000
none           0.000 50.000  0.000  0.000  0.000  0.000  0.000 50.000
phd            0.000 33.330 33.330  0.000  0.000  0.000  0.000 33.330
secondary      9.800  1.960  5.880  3.920 47.060  0.000 13.730 17.650
undergraduate  3.570 25.000  3.570  3.570 21.430 10.710 21.430 10.710
vocational    25.000  6.250  6.250  0.000 18.750  0.000 12.500 31.250


In [444]:
# Replace "NO" with "NO" and all other values with "YES"
df['q1_themes_grouped'] = df['q1_themes'].apply(lambda x: 'NO' if 'NO' in x else 'YES')

# Check the result
print(df[['q1_themes', 'q1_themes_grouped']].head())


  q1_themes q1_themes_grouped
0      [NO]                NO
1      [LA]               YES
2      [NO]                NO
3      [RA]               YES
4      [AC]               YES


In [445]:
df.to_excel('Survey_powerbi.xlsx',index=False)

In [446]:
df.columns

Index(['id', 'employment_status', 'obtained', 'gender', 'age', 'location',
       'education', 'household_income', 'duration_unemployed', 'experience',
       'q1_themes', 'q2_themes', 'assistive_tech',
       'duration_unemployed_ordinal', 'household_income_ordinal',
       'education_ordinal', 'race_African', 'race_Coloured', 'race_Indian',
       'Epilepsy', 'Physical_disability', 'Visual_impairment',
       'Mental_health_condition', 'Hearing_impairment', 'q1_themes_grouped'],
      dtype='object')

## Why do people struggle to find work?

### Logistic regression

In [447]:
df.columns

Index(['id', 'employment_status', 'obtained', 'gender', 'age', 'location',
       'education', 'household_income', 'duration_unemployed', 'experience',
       'q1_themes', 'q2_themes', 'assistive_tech',
       'duration_unemployed_ordinal', 'household_income_ordinal',
       'education_ordinal', 'race_African', 'race_Coloured', 'race_Indian',
       'Epilepsy', 'Physical_disability', 'Visual_impairment',
       'Mental_health_condition', 'Hearing_impairment', 'q1_themes_grouped'],
      dtype='object')

In [448]:
columns_for_logreg = ['employment_status', 'age', 'education_ordinal', 'household_income_ordinal', 
    'duration_unemployed_ordinal', 'Mental_health_condition', 'Hearing_impairment',
    'Visual_impairment', 'Physical_disability', 'gender',
    'race_African', 'location','race_Coloured','race_Indian', 
    'experience', 'assistive_tech',]

In [449]:
q1_df = df[columns_for_logreg]

In [450]:
q1_df.columns

Index(['employment_status', 'age', 'education_ordinal',
       'household_income_ordinal', 'duration_unemployed_ordinal',
       'Mental_health_condition', 'Hearing_impairment', 'Visual_impairment',
       'Physical_disability', 'gender', 'race_African', 'location',
       'race_Coloured', 'race_Indian', 'experience', 'assistive_tech'],
      dtype='object')

### Logistic regression

In [451]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report

# Define X (all features except employment status)
X = q1_df.drop(columns=['employment_status'])

# Define y (employment status as the target variable)
y = q1_df['employment_status']

# Scale the features before splitting into training and test sets
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)  # Scale all features

# Split into training (80%) and test (20%) sets after scaling
X_train_scaled, X_test_scaled, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=4, stratify=y)

# Initialize and fit logistic regression model
model = LogisticRegression()
model.fit(X_train_scaled, y_train)

# Predict on scaled test data
y_pred = model.predict(X_test_scaled)

# Print accuracy and classification report
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))



Accuracy: 0.9166666666666666
Classification Report:
               precision    recall  f1-score   support

           0       1.00      0.50      0.67         4
           1       0.91      1.00      0.95        20

    accuracy                           0.92        24
   macro avg       0.95      0.75      0.81        24
weighted avg       0.92      0.92      0.90        24



In [452]:
q1_df

,employment_status,age,education_ordinal,household_income_ordinal,duration_unemployed_ordinal,Mental_health_condition,Hearing_impairment,Visual_impairment,Physical_disability,gender,race_African,location,race_Coloured,race_Indian,experience,assistive_tech
0,1,29,1,0,0,0,0,1,0,0,1,1,0,0,1,0
1,1,27,1,0,2,0,0,1,0,1,1,0,0,0,0,0
2,1,45,3,3,4,0,1,0,1,0,0,1,0,1,1,1
3,1,36,1,1,1,0,0,1,0,1,0,1,0,0,1,1
4,1,25,2,0,1,0,0,0,1,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
113,0,42,2,1,2,0,0,0,1,0,1,1,0,0,1,0
114,0,31,1,0,1,0,0,0,1,1,1,0,0,0,1,0
115,0,31,1,0,2,0,0,1,0,1,1,1,0,0,0,0
116,0,36,1,0,6,0,1,0,0,0,1,1,0,0,1,1


In [453]:
import statsmodels.api as sm

# Separate target and predictors
y = q1_df['employment_status']
X = q1_df.drop('employment_status', axis=1)

# Scale age
#scaler = StandardScaler()
X['age'] = scaler.fit_transform(X[['age']])

# Add constant
X = sm.add_constant(X)

# Fit Firth logistic regression with modified parameters
model = sm.Logit(y, X).fit_regularized(
    method='l1',
    alpha=0,
    refit=True,
    trim_mode='auto',
    conv='params',
    maxiter=1000,
    cnvrg_tol=1e-8,
)

# Print summary
print(model.summary())



Optimization terminated successfully    (Exit mode 0)
            Current function value: 0.3114553985117371
            Iterations: 111
            Function evaluations: 113
            Gradient evaluations: 111
                           Logit Regression Results                           
Dep. Variable:      employment_status   No. Observations:                  118
Model:                          Logit   Df Residuals:                      102
Method:                           MLE   Df Model:                           15
Date:                Sat, 05 Apr 2025   Pseudo R-squ.:                  0.2943
Time:                        20:47:22   Log-Likelihood:                -36.752
converged:                       True   LL-Null:                       -52.080
Covariance Type:            nonrobust   LLR p-value:                  0.009765
                                  coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------

In [454]:
# Calculate odds ratios
odds_ratios = np.exp(model.params)

# Set pandas display format to avoid scientific notation
pd.set_option('display.float_format', '{:.3f}'.format)

# Display results
print(odds_ratios)

const                                    2.339
age                                      0.447
education_ordinal                        0.918
household_income_ordinal                 0.493
duration_unemployed_ordinal              1.364
Mental_health_condition                  1.539
Hearing_impairment                       1.215
Visual_impairment                        1.221
Physical_disability                      3.476
gender                                   3.221
race_African                             0.162
location                                 0.542
race_Coloured                    165409588.813
race_Indian                   339099298550.500
experience                               3.964
assistive_tech                           5.064
dtype: float64


In [455]:
# Separate target and predictors
y = q1_df['employment_status']
X = q1_df['education_ordinal']

# Add constant
X = sm.add_constant(X)

# Fit Firth logistic regression with modified parameters
model = sm.Logit(y, X).fit()

# Print summary
print(model.summary())

# Calculate odds ratios
odds_ratios = np.exp(model.params)

# Set pandas display format to avoid scientific notation
pd.set_option('display.float_format', '{:.3f}'.format)

# Display results
print(odds_ratios)                                                                              

Optimization terminated successfully.
         Current function value: 0.437146
         Iterations 6
                           Logit Regression Results                           
Dep. Variable:      employment_status   No. Observations:                  118
Model:                          Logit   Df Residuals:                      116
Method:                           MLE   Df Model:                            1
Date:                Sat, 05 Apr 2025   Pseudo R-squ.:                0.009530
Time:                        20:47:22   Log-Likelihood:                -51.583
converged:                       True   LL-Null:                       -52.080
Covariance Type:            nonrobust   LLR p-value:                    0.3191
                        coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------
const                 1.3151      0.410      3.208      0.001       0.512       2.118
educatio

In [456]:
# Create a crosstab and normalize by row
crosstab_percent = pd.crosstab(df['education'], df['employment_status'], normalize='index') * 100

# Display the result
print(crosstab_percent.round(2))



employment_status      0       1
education                       
diploma           30.000  70.000
honours           16.670  83.330
masters            6.670  93.330
none               0.000 100.000
phd                0.000 100.000
secondary         18.000  82.000
undergraduate     10.530  89.470
vocational        25.000  75.000


In [457]:
crosstab_percent = pd.crosstab(df['education'], df['q1_themes_grouped'], normalize='index') * 100

# Display the result
print(crosstab_percent)

q1_themes_grouped     NO     YES
education                       
diploma           60.000  40.000
honours           16.667  83.333
masters            0.000 100.000
none               0.000 100.000
phd                0.000 100.000
secondary         48.000  52.000
undergraduate     31.579  68.421
vocational        25.000  75.000


In [458]:
# First convert YES/NO to 1/0
df['themes_binary'] = df['q1_themes_grouped'].map({'YES': 1, 'NO': 0})

# Define dependent and independent variables
X = df[['education_ordinal']]  # Independent variable (education level)
y = df['themes_binary']        # Dependent variable (now 1 for YES, 0 for NO)

# Add constant term
X = sm.add_constant(X)

# Run logistic regression
model = sm.Logit(y, X).fit()

# Display results
print(model.summary())

Optimization terminated successfully.
         Current function value: 0.600648
         Iterations 5
                           Logit Regression Results                           
Dep. Variable:          themes_binary   No. Observations:                  118
Model:                          Logit   Df Residuals:                      116
Method:                           MLE   Df Model:                            1
Date:                Sat, 05 Apr 2025   Pseudo R-squ.:                 0.06201
Time:                        20:47:22   Log-Likelihood:                -70.877
converged:                       True   LL-Null:                       -75.562
Covariance Type:            nonrobust   LLR p-value:                  0.002204
                        coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------
const                -0.1535      0.330     -0.465      0.642      -0.801       0.494
educatio

In [459]:
odds_ratio = np.exp(model.params)

# Display odds ratio
print("Odds Ratios:")
print(odds_ratio)

Odds Ratios:
const               0.858
education_ordinal   1.401
dtype: float64


In [460]:
df_exploded['q1_themes'].value_counts(normalize=True) * 100 

q1_themes
NO    25.641
RA    21.154
DI    18.590
YES   13.462
AC     7.051
NR     5.769
LA     4.487
LC     3.846
Name: proportion, dtype: float64